In [4]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.2.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Get:4 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:11 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [73.9 kB]
Hit:13 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:14 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
G

In [5]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2021-12-14 22:53:50--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  5.73MB/s    in 0.2s    

2021-12-14 22:53:50 (5.73 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [6]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

### Load Amazon Data into Spark DataFrame

In [7]:
from pyspark import SparkFiles
url = "https://videos-reviews.s3.us-east-2.amazonaws.com/amazon_reviews_us_Video_Games_v1"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Video_Games_v1"), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   12039526| RTIS3L2M1F5SM|B001CXYMFS|     737716809|Thrustmaster T-Fl...|     Video Games|          5|            0|          0|   N|                Y|an amazing joysti...|Used this for Eli...| 2015-08-31|
|         US|    9636577| R1ZV7R40OLHKD|B00M920ND6|     569686175|Tonsee 6 buttons ...|     Video Games|          5|    

### Create DataFrames to match tables

In [8]:
from pyspark.sql.functions import to_date
# Read in the Review dataset as a DataFrame
review_df = df
review_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   12039526| RTIS3L2M1F5SM|B001CXYMFS|     737716809|Thrustmaster T-Fl...|     Video Games|          5|            0|          0|   N|                Y|an amazing joysti...|Used this for Eli...| 2015-08-31|
|         US|    9636577| R1ZV7R40OLHKD|B00M920ND6|     569686175|Tonsee 6 buttons ...|     Video Games|          5|    

In [9]:
# Drop null values
review_df = review_df.dropna()
review_df.show()
type(review_df)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   12039526| RTIS3L2M1F5SM|B001CXYMFS|     737716809|Thrustmaster T-Fl...|     Video Games|          5|            0|          0|   N|                Y|an amazing joysti...|Used this for Eli...| 2015-08-31|
|         US|    9636577| R1ZV7R40OLHKD|B00M920ND6|     569686175|Tonsee 6 buttons ...|     Video Games|          5|    

pyspark.sql.dataframe.DataFrame

In [10]:
# Create the vine_table. DataFrame
vine_df = review_df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RTIS3L2M1F5SM|          5|            0|          0|   N|                Y|
| R1ZV7R40OLHKD|          5|            0|          0|   N|                Y|
|R3BH071QLH8QMC|          1|            0|          1|   N|                Y|
|R127K9NTSXA2YH|          3|            0|          0|   N|                Y|
|R32ZWUXDJPW27Q|          4|            0|          0|   N|                Y|
|R3AQQ4YUKJWBA6|          1|            0|          0|   N|                Y|
|R2F0POU5K6F73F|          5|            0|          0|   N|                Y|
|R3VNR804HYSMR6|          5|            0|          0|   N|                Y|
| R3GZTM72WA2QH|          5|            0|          0|   N|                Y|
| RNQOY62705W1K|          4|            0|          0|   N|     

### Connect to the AWS RDS instance and write each DataFrame to its table. 

In [11]:
# Store environmental variable
from getpass import getpass
password = getpass('Enter AWS Server database password which you want to connect')

# Configure settings for RDS
mode = "append"
# with the url link ensure that the correct db name is mentioned
jdbc_url="jdbc:postgresql://review-db.cd5tiuxtyygd.us-east-2.rds.amazonaws.com:5432/vine_db"
config = {"user":"postgres", 
          "password": password, 
          "driver":"org.postgresql.Driver"}

Enter AWS Server database password which you want to connect··········


In [12]:
# Write vine_df to table in RDS
# 11 minutes
vine_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)

# Deliverable 2: Determine Bias of Vine Reviews

# 5 Star review Analysis

In [13]:
# Step 1 . retrieve all the rows where the total_votes count is equal to or greater than 20
vine_bias_df = vine_df.filter(vine_df.total_votes>=20)
vine_bias_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| R4PKAZRQJJX14|          1|           21|         34|   N|                N|
|R2CI0Y288CC7E2|          1|           21|         35|   N|                Y|
|R127WEQY2FM1T3|          1|          147|        175|   N|                Y|
|R3EZ0EPYLDA34S|          1|           14|         31|   N|                Y|
|R2FJ94555FZH32|          2|           55|         60|   N|                N|
|R1U3AR67RE273L|          1|           51|         65|   N|                Y|
|R3PZOXA5X1U8KW|          4|           31|         36|   N|                N|
| R6KTC1OPIOIIG|          2|           19|         34|   N|                Y|
|R36O341WWXXKNP|          5|           28|         31|   N|                N|
|R3GSK9MM8DNOYI|          1|            4|         32|   N|     

In [14]:
# Step 2. retrieve all the rows where the number of helpful_votes divided by total_votes is equal to or greater than 50%.
vine_helpful_df = vine_bias_df.filter(vine_bias_df.helpful_votes/vine_bias_df.total_votes>=0.5)
vine_helpful_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| R4PKAZRQJJX14|          1|           21|         34|   N|                N|
|R2CI0Y288CC7E2|          1|           21|         35|   N|                Y|
|R127WEQY2FM1T3|          1|          147|        175|   N|                Y|
|R2FJ94555FZH32|          2|           55|         60|   N|                N|
|R1U3AR67RE273L|          1|           51|         65|   N|                Y|
|R3PZOXA5X1U8KW|          4|           31|         36|   N|                N|
| R6KTC1OPIOIIG|          2|           19|         34|   N|                Y|
|R36O341WWXXKNP|          5|           28|         31|   N|                N|
|R10LZVBLQHBVJ0|          2|          151|        198|   N|                N|
|R1VR5GLGY1GE7N|          1|           49|         51|   N|     

In [15]:
# Step 3. retrieves all the rows where a review was written as part of the Vine program (paid), vine == 'Y'.
vine_program_paid = vine_helpful_df.filter(vine_helpful_df.vine=="Y")
vine_program_paid.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3KKUSGFZWSUIY|          5|           56|         63|   Y|                N|
|R10FO5UKKVZBK2|          3|           23|         23|   Y|                N|
| RM4KSGEOR7MU1|          5|           19|         24|   Y|                N|
| RG7VRMYLEXD23|          4|           22|         26|   Y|                N|
|R11O4YSCPSNL6L|          3|           20|         26|   Y|                N|
|R286MFBAJ8NPD6|          5|           46|         51|   Y|                N|
|R1JRR530H4COA2|          5|           22|         28|   Y|                N|
| RQ5WD90PUNBU9|          5|           21|         24|   Y|                N|
|R12648VHCQWUV9|          4|           21|         28|   Y|                N|
|R3KAW29CJ8L6DQ|          5|           17|         20|   Y|     

In [16]:
vine_program_paid.count()

94

In [17]:
# Step 4. retrieve all the rows where the review was not part of the Vine program (unpaid), vine == 'N'.
vine_program_unpaid = vine_helpful_df.filter(vine_helpful_df.vine=="N")
vine_program_unpaid.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| R4PKAZRQJJX14|          1|           21|         34|   N|                N|
|R2CI0Y288CC7E2|          1|           21|         35|   N|                Y|
|R127WEQY2FM1T3|          1|          147|        175|   N|                Y|
|R2FJ94555FZH32|          2|           55|         60|   N|                N|
|R1U3AR67RE273L|          1|           51|         65|   N|                Y|
|R3PZOXA5X1U8KW|          4|           31|         36|   N|                N|
| R6KTC1OPIOIIG|          2|           19|         34|   N|                Y|
|R36O341WWXXKNP|          5|           28|         31|   N|                N|
|R10LZVBLQHBVJ0|          2|          151|        198|   N|                N|
|R1VR5GLGY1GE7N|          1|           49|         51|   N|     

In [18]:
vine_program_unpaid.count()

40471

In [47]:
# Step 5. the number of 5-star reviews, and the percentage of 5-star reviews for the two types of review (paid vs unpaid)
# function to calculate the number and percentage of 5 star reviews
def summary_five_star(eval_df):
  count_five_star = eval_df.where(eval_df.star_rating==5).count()
  total_reviews = eval_df.count()
  percentage_five_star = (count_five_star/total_reviews)*100
  return count_five_star, total_reviews,percentage_five_star

In [50]:
# Step 5. calculate the totals and percentage for both paid and unpaid.
count_five_star_paid, total_five_star_paid, percentage_five_star_paid = summary_five_star(vine_program_paid)
count_five_star_unpaid, total_five_star_unpaid, percentage_five_star_unpaid = summary_five_star(vine_program_unpaid)
count_five_star = count_five_star_paid + count_five_star_unpaid

In [51]:
count_five_star_total, total_reviews, percentage_five_star_total = summary_five_star(vine_df)
print(f"The total number of 5 star reviews for all vine reviews without any filters is {count_five_star_total} out of Total reviews {total_reviews} and its percentage is {percentage_five_star_total}")

The total number of 5 star reviews for all vine reviews without any filters is 1026856 out of Total reviews 1785886 and its percentage is 57.498406953187384


In [52]:
count_five_star_helpful, total_five_star_helpful ,percentage_five_star_helpful = summary_five_star(vine_helpful_df)
print(f"The total number of 5 star reviews for all vine reviews where total votes is >20 and the the number of helpful_votes divided by total_votes is equal to or greater than 50% is {count_five_star_helpful} out of {total_five_star_helpful} and percentage five star is {percentage_five_star_helpful}.")

The total number of 5 star reviews for all vine reviews where total votes is >20 and the the number of helpful_votes divided by total_votes is equal to or greater than 50% is 15711 out of 40565 and percentage five star is 38.73043263897449.


In [53]:
# Step 5. Print the results of the above calculations 
print(f"The total number of 5 star reviews for all vine reviews above 20 and greater thab 50% is {count_five_star}.")
print(f"The total number of 5 star reviews for paid vine reviews above 20 and greater thab 50% is {count_five_star_paid} out of {total_five_star_paid} and The percentage of the paid 5 star vine reviews is {percentage_five_star_paid}.")
print(f"The total number of 5 star reviews for unpaid vine reviews above 20 and greater thab 50% is {count_five_star_unpaid} out of {total_five_star_unpaid} and The percentage of the unpaid 5 star vine reviews is {percentage_five_star_unpaid}.")

The total number of 5 star reviews for all vine reviews above 20 and greater thab 50% is 15711.
The total number of 5 star reviews for paid vine reviews above 20 and greater thab 50% is 48 out of 94 and The percentage of the paid 5 star vine reviews is 51.06382978723404.
The total number of 5 star reviews for unpaid vine reviews above 20 and greater thab 50% is 15663 out of 40471 and The percentage of the unpaid 5 star vine reviews is 38.701786464381904.


# Verified purchase and Non verified purchase reviews Analysis

In [49]:
# Step 5. the number of 5-star reviews, and the percentage of 5-star reviews for the two types of review (paid vs unpaid)
def summary_five_star_additional(eval_df, verified_status):
  count_five_star = 0
  percentage_five_star = 0
  if verified_status:
    eval_df = eval_df.filter(eval_df.verified_purchase=="Y")
  else:
    eval_df = eval_df.filter(eval_df.verified_purchase=="N")
  if eval_df.count():
    count_five_star = eval_df.where(eval_df.star_rating==5).count()
    total_reviews = eval_df.count()
    percentage_five_star = (count_five_star/total_reviews)*100
  return count_five_star, percentage_five_star

In [27]:
# Step 5. calculate the totals and percentage for both paid and unpaid.
count_five_star_paid_verified, percentage_five_star_paid_verified = summary_five_star(vine_program_paid, True)
count_five_star_unpaid_verified, percentage_five_star_unpaid_verified = summary_five_star(vine_program_unpaid, True)

In [28]:
# Step 5. Print the results of the above calculations
print(f"The total number of 5 star reviews for paid and verified purchase vine reviews is {count_five_star_paid_verified} and The percentage of the paid and verified purchase 5 star vine reviews is {percentage_five_star_paid_verified}.")
print(f"The total number of 5 star reviews for unpaid and verified purchase vine reviews is {count_five_star_unpaid_verified} and The percentage of the unpaid purchase and verified 5 star vine reviews is {percentage_five_star_unpaid_verified}.")

The total number of 5 star reviews for paid and verified purchase vine reviews is 0 and The percentage of the paid and verified purchase 5 star vine reviews is 0.
The total number of 5 star reviews for unpaid and verified purchase vine reviews is 3862 and The percentage of the unpaid purchase and verified 5 star vine reviews is 38.21492182861667.


In [29]:
count_five_star_paid_unverified, percentage_five_star_paid_unverified = summary_five_star(vine_program_paid, False)
count_five_star_unpaid_unverified, percentage_five_star_unpaid_unverified = summary_five_star(vine_program_unpaid, False)

In [31]:
# Step 5. Print the results of the above calculations
print(f"The total number of 5 star reviews for paid and unverified purchase reviews is {count_five_star_paid_unverified} and The percentage of the paid and unverified purchase 5 star vine reviews is {percentage_five_star_paid_unverified}.")
print(f"The total number of 5 star reviews for unpaid and unverified purchase vine reviews is {count_five_star_unpaid_unverified} and The percentage of the unpaid and unverified purchase 5 star vine reviews is {percentage_five_star_unpaid_unverified}.")

The total number of 5 star reviews for paid and unverified purchase reviews is 48 and The percentage of the paid and unverified purchase 5 star vine reviews is 51.06382978723404.
The total number of 5 star reviews for unpaid and unverified purchase vine reviews is 11801 and The percentage of the unpaid and unverified purchase 5 star vine reviews is 38.86382348098139.


In [33]:
import pandas as pd

In [44]:
analysis_df = pd.DataFrame([[count_five_star_paid, percentage_five_star_paid, count_five_star_paid_verified,percentage_five_star_paid_verified,count_five_star_paid_unverified,percentage_five_star_paid_unverified],[count_five_star_unpaid, percentage_five_star_unpaid, count_five_star_unpaid_verified,percentage_five_star_unpaid_verified,count_five_star_unpaid_unverified,percentage_five_star_unpaid_unverified]])
analysis_df.columns = ['Total Count Five star','Total Percentage five star','Total 5 star Verified','Percentage 5 star verified','Total 5 star Unverified','Percentage 5 star Unverified']
analysis_df.index = ['Paid', 'Unpaid']


In [45]:
analysis_df

,Total Count Five star,Total Percentage five star,Total 5 star Verified,Percentage 5 star verified,Total 5 star Unverified,Percentage 5 star Unverified
Paid,48,51.063830,0,0.000000,48,51.063830
Unpaid,15663,38.701786,3862,38.214922,11801,38.863823
